## データ前処理についての勉強
よりよい訓練データセットの構築について学ぶ

#### 欠測データへの対処
欠測データは予期せぬ結果を生み出す原因となる。

In [3]:
import pandas as pd
from io import StringIO
# サンプルデータの作成
csv_data =  '''A,B,C,D
            1.0,2.0,3.0,4.0
            5.0,6.0,,8.0
            10.0,11.0,12.0,'''
# CSVデータを読み込む
df = pd.read_csv(StringIO(csv_data))
print(df)


      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN


##### 手法1 欠測データをもつ訓練データ/特徴量を削除

In [5]:
# 欠測値を含む行を削除
print(df.dropna())
# 欠測値を含む列を削除
print(df.dropna(axis = 1))
# すべての列がNaNである行を削除
print(df.dropna(how = 'all'))
# 非NaN値が4つ未満の行を削除
print(df.dropna(thresh = 4))
# 特定の列にNaNがある行を削除
print(df.dropna(subset = ['C']))

     A    B    C    D
0  1.0  2.0  3.0  4.0
      A     B
0   1.0   2.0
1   5.0   6.0
2  10.0  11.0
      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN
     A    B    C    D
0  1.0  2.0  3.0  4.0
      A     B     C    D
0   1.0   2.0   3.0  4.0
2  10.0  11.0  12.0  NaN


データ点を削除すると解析の信頼性が失われる他、特徴量の列を削除しすぎると有益な情報を減らすことになる。一般に手法1は現実的ではない。
##### 手法2 補完法

以下は
sklearn.imputeモジュールからSimpleImputerクラスをインポートし、欠損値補完のインスタンスimrを生成。missing_values引数には、欠損値として扱う値を指定している。ここでは、np.nanを欠損値として扱うようにしている。また、strategy引数には、欠損値を補完する方法を指定している。ここでは、平均値を用いて欠損値を補完するようにしている。

欠損値補完のインスタンスimrに、Pandas DataFrameのdfをNumPy配列に変換したデータdf.valuesを入力して、欠損値補完のモデルを学習。fitメソッドは、入力されたデータを用いて欠損値補完のモデルを学習するメソッド。

学習済みのモデルimrを用いて、df.valuesの欠損値を平均値で補完し、補完後のデータをimputed_dataに代入する。transformメソッドは、学習済みのモデルを用いてデータを変換するメソッドであり、補完後のデータを返す。


In [6]:
"""平均値補完"""
from sklearn.impute import SimpleImputer
import numpy as np
# 欠測値補完のインスタンスを生成
imr = SimpleImputer(missing_values = np.nan, strategy = 'mean') #np.nanは欠損値を表す。その列の平均値で補完、最頻値はmost_frequent
# データを適合
imr = imr.fit(df.values) #df.valuesでnumpy配列に変換し、fitで学習
# 補完を実行
imputed_data = imr.transform(df.values) #transformはfitしたデータを変換する
print(imputed_data)


[[ 1.   2.   3.   4. ]
 [ 5.   6.   7.5  8. ]
 [10.  11.  12.   6. ]]


scikit-learnには変換器クラスと推定器クラスがある。
変換器クラスは、データの前処理や特徴量エンジニアリングなどのタスクを実行するためのクラス。具体的には、データの欠損値や異常値の処理、正規化、特徴量の選択や抽出、次元削減などのタスクを実行することができる。変換器は、fit()メソッドで変換器自身を学習し、transform()メソッドでデータを変換する。

推定器クラスは、データを学習し、予測するためのクラス。具体的には、教師あり学習の場合は入力データと対応するラベルデータを用いて、モデルを学習し、未知のデータに対して予測値を出力することができる。一方、教師なし学習の場合は、入力データだけを用いて、データの構造やパターンを解析し、その情報を用いてクラスタリングや次元削減などのタスクを実行することができる。推定器は、fit()メソッドを用いてデータを学習し、predict()メソッドを用いて予測を行う。
`前章でやったSVMやロジスティック回帰がこれにあたる。`

##### 数値データではなくカテゴリデータを扱う方法
カテゴリデータとは、Tシャツでいう青とか赤みたいな感じ。

In [8]:
import pandas as pd
# サンプルデータの作成(Tシャツの色・サイズ・価格・クラスラベル)
df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                     ['red', 'L', 13.5, 'class2'],
                        ['blue', 'XL', 15.3, 'class1']])
# 列名を設定
df.columns = ['color', 'size', 'price', 'classlabel']
print(df)


   color size  price classlabel
0  green    M   10.1     class1
1    red    L   13.5     class2
2   blue   XL   15.3     class1


sizeみたいな順序特徴量を正しく解釈させるには、文字列の値を整数に対応させる必要がある。

In [9]:
# Tシャツのサイズと整数を対応させる辞書を作成
size_mapping = {'XL': 3, 'L': 2, 'M': 1}
# Tシャツのサイズを整数に変換
df['size'] = df['size'].map(size_mapping) #mapメソッドは辞書のキーに対応する値を取り出す。要はキーと値を入れ替える
print(df)

   color  size  price classlabel
0  green     1   10.1     class1
1    red     2   13.5     class2
2   blue     3   15.3     class1


In [10]:
"""あとから整数値を元の文字列表現に戻したい場合"""
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping) #変換された特徴量の列でmapメソッドを呼び出す

0     M
1     L
2    XL
Name: size, dtype: object

#### クラスラベルのエンコーディング
多くの機械学習ライブラリではクラスラベルは整数値としてエンコードされている必要がある。クラスラベルは順序は特にないから、0から順に番号付けすればok

In [21]:
import numpy as np
# クラスラベルと整数を対応させる辞書を作成
class_mapping = {label: idx for idx, label in 
                enumerate(np.unique(df['classlabel']))} #enumerateはインデックスと要素を取り出す(for文だけど、タプルで返す)
# np.uniqueは重複を除いた要素を取り出す
# for idx, label in enumerate(np.unique(df['classlabel']))
print(class_mapping)
#print('\n')
# クラスラベルを整数に変換
df['classlabel'] = df['classlabel'].map(class_mapping) #mapメソッドは辞書のキーに対応する値を取り出す。要はキーと値を入れ替える
print(df)


{0: 0, 1: 1}
   color  size  price  classlabel
0  green     1   10.1           0
1    red     2   13.5           1
2   blue     3   15.3           0


In [20]:
#整数からクラスラベルに変換する逆の辞書を作成
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


上の例では、なぜかclasslabelが最初から0, 1に自動変換されている。{class1: 0, class2: 1}となるはず。だから逆の辞書を作って元に戻すというコードだけど戻っていない
####  次のコードはクラスラベルを整数変換するための便利なクラスを使っている。

In [23]:
from sklearn.preprocessing import LabelEncoder
# ラベルエンコーダのインスタンスを生成
class_le = LabelEncoder()
# クラスラベルを整数に変換
y = class_le.fit_transform(df['classlabel'].values) #fit_transformはfitとtransformをまとめたもの
print(y)

#クラスラベルを元の文字列に戻す (今回の例では文字列がすでに整数に変換されているので、元に戻すことはできない)
class_le.inverse_transform(y)

[0 1 0]


array([0, 1, 0])

#### ↖︎今まで使用したデータフレームについておさらい
```python
import pandas as pd
# サンプルデータの作成(Tシャツの色・サイズ・価格・クラスラベル)
df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                     ['red', 'L', 13.5, 'class2'],
                        ['blue', 'XL', 15.3, 'class1']])
# 列名を設定
df.columns = ['color', 'size', 'price', 'classlabel']
print(df)
```
サイズは順序特徴量なので、M, L, XLの順で1, 2, 3と変換した。クラスラベルは順序を持たないので、LabelEncoderを使って整数に変換した。では色について、blue→0, green→1, red→2と変換して良いのか？

#### 答えは「NO」
色には順序はないけれども、学習アルゴリズムはgreenが0よりも大きく、redがgreenよりも大きいと想定してしまう。これは正しくない。
これを避けるために`「one-hotエンコーティング」`という手法をつかう。

In [24]:
from sklearn.preprocessing import OneHotEncoder
"""このメソッドを使用することで、pandasのデータフレームやシリーズオブジェクトに含まれるデータを、
NumPyの多次元配列として扱うことができます。NumPyの配列は、高速な数値演算を行うことができるため、
データの加工や解析において、非常に有用なデータ形式となります。
"""
X = df[['color', 'size', 'price']].values #valuesはnumpy配列に変換
# one-hotエンコーダのインスタンスを生成
color_ohe = OneHotEncoder()
# one-hotエンコーディングを実行
"""reshape()関数は、NumPyの配列（array）オブジェクトに対して、その形状を変更するためのメソッドです。
具体的には、reshape()関数をNumPyの配列に適用すると、その配列の要素を、指定した形状に変形することができます。
たとえば、1次元の配列を2次元の行列に変形することができます。このとき、変形後の行列の要素数が元の配列の要素数と一致するように指定する必要があります。
reshape()関数は、元の配列自体を変更するのではなく、新しい配列を作成します。
そのため、元の配列の形状を変更する場合でも、元の配列自体を維持したまま操作することができます。
また、reshape()関数を用いて配列の形状を変更することで、配列のデータを別の形式で解釈することができます。
たとえば、1次元の配列を2次元の行列に変形することで、その配列を画像のピクセルデータとして解釈することができます。
"""
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray() #X[:, 0]は0列目のcolorのこと、reshape(-1, 1)は1列の2次元配列に変換
#toarray()はnumpy配列に変換

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

上のコードで利用しているreshapeメソッド、reshape(1, -1)で１次元配列を２次元1行の配列に変換する。reshape(-1, 1)で１次元配列を２次元１列の配列にする。
 ```python
 import numpy as np
np.arange(6).reshape((1, -1))
np.arrange(6).reshape((-1, 1))
```
出力
```python
array([[0, 1, 2, 3, 4, 5]])
array([[0],
       [1],
       [2],
       [3],
       [4],
       [5]])
```


下のコードは、複数特徴量がある場合に、one-hotエンコーディングをする特徴量を選択して変換できるやり方
俺的にはこっちの方がありがたいかも

In [28]:
"""from sklearn.compose import ColumnTransformer
X = df[['color', 'size', 'price']].values #valuesはnumpy配列に変換
c_transf = ColumnTransformer([('onehot', OneHotEncoder(), [0]), 
                              ('nothing', 'passthrough', '[1, 2]')])#0列目のcolorをonehotエンコーディング、1,2列目のsize,priceはそのまま
c_transf.fit_transform(X) #astype(float)はfloat型に変換
"""
#上はエラーが出るので、下のようにする
# get_dummies関数は文字列値を持つ列のみをonehotエンコーディングできる
pd.get_dummies(df[['price', 'color', 'size']]) #pandasのget_dummies関数でもonehotエンコーディングができる




,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


one-hotエンコーディングは多重共線性を招き、正しい予測ができなくなる可能性もある。そんなときは特徴量を１つ削除し、相関を減らす。

In [30]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True) #drop_first=Trueでダミー変数の最初の列を削除する

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0
